<a href="https://colab.research.google.com/github/GIJOE1003/3103ETF/blob/main/Notebook3_Volatility_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETF Volatility Forecasting

This notebook forecasts 7-day volatility using multiple regression models.

In [1]:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping


## Data Collection

In [3]:

# Data Collection
tickers = ['SPY', 'QQQ', 'GLD','TLT', 'VTI','EEM','XLF','XLV','VEA','VNQ']
end_date = pd.to_datetime('today')
start_date = end_date - pd.DateOffset(months=120)

# Download prices
df_raw = yf.download(tickers, start=start_date, end=end_date)

df_open = df_raw['Open'].reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Open')
df_close = df_raw['Close'].reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Close')
df_high  = df_raw['High'].reset_index().melt(id_vars='Date', var_name='Ticker', value_name='High')
df_low   = df_raw['Low'].reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Low')
df_volume = df_raw['Volume'].reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Volume')

# Merge into one DataFrame
df = df_open.merge(df_close, on=['Date', 'Ticker']).merge(df_high, on=['Date', 'Ticker']).merge(df_low, on=['Date', 'Ticker']).merge(df_volume, on=['Date', 'Ticker'])
df = df.round(2)

# Create CSV File
df.to_csv("combined_etf_data.csv", index = False)
print(f"Combined data saved: {df.shape}")
print(df.head())
print(df.tail())

df.isnull().sum()
df.info()


[*********************100%***********************]  10 of 10 completed


Combined data saved: (25160, 7)
        Date Ticker   Open  Close   High    Low    Volume
0 2015-04-10    EEM  34.21  34.38  34.39  34.18  49899000
1 2015-04-13    EEM  34.45  34.18  34.57  34.17  55031100
2 2015-04-14    EEM  34.26  34.34  34.41  34.11  42601100
3 2015-04-15    EEM  34.33  34.58  34.59  34.26  40254700
4 2015-04-16    EEM  34.67  34.85  35.05  34.62  55057700
            Date Ticker    Open   Close    High     Low    Volume
25155 2025-04-03    XLV  143.76  143.13  145.19  143.05  10990600
25156 2025-04-04    XLV  141.14  135.28  141.71  135.23  21341100
25157 2025-04-07    XLV  131.71  134.47  137.37  129.66  29940300
25158 2025-04-08    XLV  139.43  132.98  139.44  131.28  20860400
25159 2025-04-09    XLV  130.13  138.76  139.24  129.68  39441700
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25160 entries, 0 to 25159
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    25160 non-null 

## Feature Engineering and Volatility Target

In [4]:
df['Return'] = df.groupby('Ticker')['Close'].pct_change().round(4)
df['Return'] = df['Return'].fillna(0.0)
df['Volatility7'] = df.groupby('Ticker')['Return'].rolling(window=20).std().reset_index(level=0, drop=True).round(4)
df['Volatility7'] = df['Volatility7'].fillna(0.0)


df['MA5'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(20).mean()).round(4)
df['Close_MA5_diff'] = ((df['Close'] - df['MA5']) / df['MA5'] * 100).round(4)

windows = [20, 50, 200]
for w in windows:
    df[f'SMA{w}'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(w).mean()).round(4)
    df[f'EMA{w}'] = df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=w, adjust=False).mean()).round(4)


df['PrevClose'] = df.groupby('Ticker')['Close'].shift(1)
df['TR'] = df[['High', 'Low', 'PrevClose']].apply(lambda x: max(x[0]-x[1], abs(x[0]-x[2]), abs(x[1]-x[2])), axis=1).round(4)
df['ATR7'] = df.groupby('Ticker')['TR'].transform(lambda x: x.rolling(7).mean()).round(4)
df['ATR7_pct'] = (df['ATR7'] / df['Close'] * 100).round(4)

def compute_rsi(series, window=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI14'] = df.groupby('Ticker')['Close'].transform(compute_rsi)

ema_12 = df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
ema_26 = df.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())

df['MACD'] = (ema_12 - ema_26).round(4)
df['Signal_Line'] = df.groupby('Ticker')['MACD'].transform(lambda x: x.ewm(span=9, adjust=False).mean()).round(4)

low14 = df.groupby('Ticker')['Low'].transform(lambda x: x.rolling(14).min())
high14 = df.groupby('Ticker')['High'].transform(lambda x: x.rolling(14).max())

df['%K'] = ((df['Close'] - low14) / (high14 - low14) * 100).round(2)
df['%D'] = df.groupby('Ticker')['%K'].transform(lambda x: x.rolling(3).mean())

df['DayOfWeek'] = pd.to_datetime(df['Date']).dt.dayofweek
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['IsMonthEnd'] = pd.to_datetime(df['Date']).dt.is_month_end.astype(int)


df.to_csv("combined_etf_data_features.csv", index = False)
print(f"Combined data saved: {df.shape}")

<ipython-input-4-c7f3bfe94bce>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['TR'] = df[['High', 'Low', 'PrevClose']].apply(lambda x: max(x[0]-x[1], abs(x[0]-x[2]), abs(x[1]-x[2])), axis=1).round(4)


Combined data saved: (25160, 29)


In [8]:
df['Return'] = df.groupby('Ticker')['Close'].pct_change()
df['Rolling_Std_7'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(7).std().shift(-7))
df.dropna(inplace=True)

features = ['High', 'Low', 'Volume','Return',
    'MA5', 'Close_MA5_diff', 'SMA20', 'EMA20','Volatility7',
    'SMA50', 'EMA50', 'SMA200', 'EMA200', 'TR', 'ATR7','RSI14','MACD','Signal_Line','%K','%D','DayOfWeek','Month','IsMonthEnd']
X = df[features]
y = df['Rolling_Std_7']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)


### Linear Regression

In [6]:

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
mae_lr = mean_absolute_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
print(f"Linear Regression - RMSE: {rmse_lr:.4f}, MAE: {mae_lr:.4f}, R2: {r2_lr:.4f}")


Linear Regression - RMSE: 0.4151, MAE: 0.2772, R2: 0.5020


### Random Forest Regression

In [7]:

rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print(f"Random Forest - RMSE: {rmse_rf:.4f}, MAE: {mae_rf:.4f}, R2: {r2_rf:.4f}")


Random Forest - RMSE: 0.4400, MAE: 0.3147, R2: 0.4406


### XGBoost Regression

In [9]:

xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"XGBoost - RMSE: {rmse_xgb:.4f}, MAE: {mae_xgb:.4f}, R2: {r2_xgb:.4f}")


XGBoost - RMSE: 0.4172, MAE: 0.2937, R2: 0.4722


### LSTM Regression

In [10]:

# Rebuild sequence data for LSTM
def create_lstm_sequences(data, target, lookback=7):
    X_lstm, y_lstm = [], []
    for ticker in data['Ticker'].unique():
        df_ticker = data[data['Ticker'] == ticker].reset_index(drop=True)
        for i in range(lookback, len(df_ticker) - 7):
            X_lstm.append(df_ticker[features].iloc[i-lookback:i].values)
            y_lstm.append(df_ticker[target].iloc[i])
    return np.array(X_lstm), np.array(y_lstm)

X_lstm, y_lstm = create_lstm_sequences(df, 'Rolling_Std_7')
X_train_lstm, X_test_lstm = X_lstm[:int(0.8*len(X_lstm))], X_lstm[int(0.8*len(X_lstm)):]
y_train_lstm, y_test_lstm = y_lstm[:int(0.8*len(y_lstm))], y_lstm[int(0.8*len(y_lstm)):]

lstm_model = Sequential([
    LSTM(64, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=32, validation_split=0.1, callbacks=[early_stop])
y_pred_lstm = lstm_model.predict(X_test_lstm)

rmse_lstm = np.sqrt(mean_squared_error(y_test_lstm, y_pred_lstm))
mae_lstm = mean_absolute_error(y_test_lstm, y_pred_lstm)
r2_lstm = r2_score(y_test_lstm, y_pred_lstm)
print(f"LSTM - RMSE: {rmse_lstm:.4f}, MAE: {mae_lstm:.4f}, R2: {r2_lstm:.4f}")


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


515/515 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 4.7552 - val_loss: 1.9368
Epoch 2/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 3.5547 - val_loss: 1.9712
Epoch 3/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 3.5515 - val_loss: 1.9209
Epoch 4/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 3.4979 - val_loss: 2.0200
Epoch 5/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3.5720 - val_loss: 1.9601
Epoch 6/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 3.5151 - val_loss: 2.0539
Epoch 7/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 3.6992 - val_loss: 2.0341
Epoch 8/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 3.5192 - val_loss: 1.8986
Epoch 9/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 3.6155 - val_loss: 1.9973
Epoch 10/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 3.5915 - val_loss: 1.9243
143/143 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
LSTM - RMSE: 1.1209, MAE: 1.0426, R2: -2.7924


In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Train models
lr_model = LinearRegression().fit(X_train, y_train)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)
xgb_model = XGBRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)

# Predict
y_pred_lr = lr_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)

# Simple average ensemble
y_pred_ensemble = (y_pred_lr + y_pred_rf + y_pred_xgb) / 3

# Evaluate
rmse = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))
mae = mean_absolute_error(y_test, y_pred_ensemble)
r2 = r2_score(y_test, y_pred_ensemble)

print(f"Ensemble Volatility Forecasting")
print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")


📊 Ensemble Volatility Forecasting
✅ RMSE: 0.3897, MAE: 0.2761, R²: 0.5394


## Final Comparison & Best Model Selection

In [13]:

results = {
    'Linear Regression': (rmse_lr, mae_lr, r2_lr),
    'Random Forest': (rmse_rf, mae_rf, r2_rf),
    'XGBoost': (rmse_xgb, mae_xgb, r2_xgb),
    'LSTM': (rmse_lstm, mae_lstm, r2_lstm),
    'Ensemble': (rmse, mae, r2)
}

df_results = pd.DataFrame(results, index=['RMSE', 'MAE', 'R2']).T.sort_values(by='RMSE')
print(df_results)


                       RMSE       MAE        R2
Ensemble           0.389702  0.276082  0.539414
Linear Regression  0.415144  0.277158  0.501959
XGBoost            0.417188  0.293726  0.472153
Random Forest      0.439976  0.314665  0.440596
LSTM               1.120897  1.042584 -2.792351


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

# Step before training
df['Rolling_STD_7'] = (
    df.sort_values(['Ticker', 'Date'])
      .groupby('Ticker')['Return']
      .rolling(window=7)
      .std()
      .shift(-7)
      .reset_index(level=0, drop=True)
)

#
forecast_days = 7
today = datetime.today().date()

# Generate next 7 days
future_dates = []
while len(future_dates) < forecast_days:
    today+= timedelta(days=1)
    if today.weekday() < 5:  # Mon–Fri
        future_dates.append(today)

# Prepare latest data
latest_df = df.groupby("Ticker").tail(1).copy().reset_index(drop=True)
future_df = pd.DataFrame(np.repeat(latest_df.values, forecast_days, axis=0), columns=latest_df.columns)
future_df["Date"] = future_dates * len(latest_df)


for col in features:
    if pd.api.types.is_numeric_dtype(future_df[col]):
        std_dev = future_df[col].std()
        if std_dev > 0:
            future_df[col] += np.random.normal(0, 0.01 * std_dev, size=len(future_df))



# Features
features = ['High', 'Low', 'Return', 'Volume',
    'MA5', 'Close_MA5_diff', 'SMA20', 'EMA20', 'Volatility7',
    'SMA50', 'EMA50', 'SMA200', 'EMA200', 'TR', 'ATR7',
    'RSI14', 'MACD', 'Signal_Line', '%K', '%D',
    'DayOfWeek', 'Month', 'IsMonthEnd']


scaler = StandardScaler()
X_full = scaler.fit_transform(df[features])
X_future = scaler.transform(future_df[features])

X_future += np.random.normal(0, 0.01, X_future.shape)

# Train base models
y = df['Rolling_STD_7']  # Your volatility target


# Filter rows where target is not NaN
valid_rows = ~y.isna()
X_full_clean = X_full[valid_rows]
y_clean = y[valid_rows]

model_lr = LinearRegression().fit(X_full_clean, y_clean)
model_rf = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_full_clean, y_clean)
model_xgb = XGBRegressor(n_estimators=100, random_state=42).fit(X_full_clean, y_clean)


# Predict
pred_lr = model_lr.predict(X_future)
pred_rf = model_rf.predict(X_future)
pred_xgb = model_xgb.predict(X_future)

# Simple average ensemble
future_df['Predicted_Volatility'] = (pred_lr + pred_rf + pred_xgb) / 3

# Output result
final_output = future_df[['Ticker', 'Date', 'Predicted_Volatility']]
print(final_output)

# Optionally export
final_output.to_csv("volatility.csv", index=False)
